In [2]:
from pymatgen.core import Composition, Element, Structure
from pymatgen.ext.matproj import MPRester
import pymatgen.core as mg

import re
import pprint
import pickle
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas
from ast import literal_eval

import multiprocessing as multi
from joblib import Memory
import pickle

import parmap

mpr = MPRester('WgPaLVntnoS3X3tdlT')
mg.__version__

'2022.0.16'

# MP에서 Li 포함 3원소 4원소 모든 물질 query

In [ ]:
data = mpr.query({"elements":'Li','nelements':{ "$gte" : 3 , "$lte" : 4}}, properties=['material_id',"pretty_formula",'elements','formula','cif'])
# MP 에서 원하는 데이터 가져옴
df = pd.DataFrame(data)
df.to_csv('./new_order_data/01_1_all_Li_material_from_MP.csv',index=False)

In [3]:
df = pd.read_csv('../new_order_data/01_1_all_Li_material_from_MP.csv')[:10]

In [4]:
df

,material_id,pretty_formula,elements,formula,cif
0,mp-1096051,LiCd2Cu,"['Cd', 'Cu', 'Li']","{'Li': 1.0, 'Cd': 2.0, 'Cu': 1.0}",# generated using pymatgen\ndata_LiCd2Cu\n_sym...
1,mp-1097065,Li2HfN2,"['Hf', 'Li', 'N']","{'Li': 2.0, 'Hf': 1.0, 'N': 2.0}",# generated using pymatgen\ndata_Li2HfN2\n_sym...
2,mp-1176979,Li6FeS4,"['Fe', 'Li', 'S']","{'Li': 6.0, 'Fe': 1.0, 'S': 4.0}",# generated using pymatgen\ndata_Li6FeS4\n_sym...
3,mp-1187090,Sr2LiRh,"['Li', 'Rh', 'Sr']","{'Sr': 2.0, 'Li': 1.0, 'Rh': 1.0}",# generated using pymatgen\ndata_Sr2LiRh\n_sym...
4,mp-1197196,Li2Mo15Se19,"['Li', 'Mo', 'Se']","{'Li': 2.0, 'Mo': 15.0, 'Se': 19.0}",# generated using pymatgen\ndata_Li2Mo15Se19\n...
5,mp-1223047,Li2UBr6,"['Br', 'Li', 'U']","{'Li': 2.0, 'U': 1.0, 'Br': 6.0}",# generated using pymatgen\ndata_Li2UBr6\n_sym...
6,mp-1246806,Li17Cl5O6,"['Cl', 'Li', 'O']","{'Li': 17.0, 'Cl': 5.0, 'O': 6.0}",# generated using pymatgen\ndata_Li17Cl5O6\n_s...
7,mp-15788,LiYS2,"['Li', 'Y', 'S']","{'Li': 1.0, 'Y': 1.0, 'S': 2.0}",# generated using pymatgen\ndata_LiYS2\n_symme...
8,mp-19013,LiV2O5,"['Li', 'O', 'V']","{'Li': 1.0, 'O': 5.0, 'V': 2.0}",# generated using pymatgen\ndata_LiV2O5\n_symm...
9,mp-754811,Li3Mn5O8,"['Li', 'Mn', 'O']","{'Li': 3.0, 'Mn': 5.0, 'O': 8.0}",# generated using pymatgen\ndata_Li3Mn5O8\n_sy...


## formula 형태 변환 ( dict --> list )

In [5]:
for i in tqdm(range(len(df))): 
    formula_dict = literal_eval(df.iloc[i]['formula'])
    df.iloc[i]['formula'] = (list(formula_dict.keys())+list(formula_dict.values()))

  0%|          | 0/10 [00:00<?, ?it/s]

## 치환에 포함할 원소 list ( HHI < 6000 ), O 제외 Li & S 포함 총 27가지

In [6]:
sub_el_list = ['Li','B','C','N','O','F','Na','Mg','Al','Si','P','S',
               'Cl','Ca','Zn','Ga','Ge','As','Se','Sr','Cd','In',
               'Sn','Te','I','Ba','Hg','Pb'] # 치환할 27개 원소

## 원소 치환

In [7]:
def substitution(df,sub_elements):
    substituted_list = []
    for j in tqdm(range(len(df))):
        temp_list = df['formula'][j]
        temp_range = list(range(int(len(temp_list)/2))) 
        del temp_range[temp_list.index('Li')] # Li 가 아닌 element 의 index 선정 ex) ['Li', 'Cd', 'Cu', 1.0, 2.0, 1.0] -> [1, 2]
        
        for i in temp_range:
            temp_range2 = temp_range[:]
            temp_list2 = temp_list[:]
            temp_list2[i]= 'S'
            temp_range2.remove(i) # S 로 먼저 하나 치환, 치환한 index 제외
            for k in temp_range2:
                temp_list3 = temp_list2[:]
                temp_range2.remove(k) 
                for s1 in sub_el_list:
                    temp_list3[k] = s1 # 나머지 28가지로 치환

                    if temp_range2: # 4원소인 경우 바로 k 있는 for문 2번 돌아감
                        for s2 in sub_el_list:
                            temp_list4 = temp_list3[:]
                            temp_list4[temp_range2[0]] = s2
                            substituted_list.append(temp_list4+[df['material_id'][j]]) # 나머지 28가지로 치환
                    else: # 3원소인 경우
                        temp_list5 = temp_list3[:]
                        substituted_list.append(temp_list5+[df['material_id'][j]])
                        
    return substituted_list

In [8]:
substituted_list=substitution(df,sub_el_list)

  0%|          | 0/10 [00:00<?, ?it/s]

In [9]:
substituted_list[:5]

[['Li', 'S', 'Li', 1.0, 2.0, 1.0, 'mp-1096051'],
 ['Li', 'S', 'B', 1.0, 2.0, 1.0, 'mp-1096051'],
 ['Li', 'S', 'C', 1.0, 2.0, 1.0, 'mp-1096051'],
 ['Li', 'S', 'N', 1.0, 2.0, 1.0, 'mp-1096051'],
 ['Li', 'S', 'O', 1.0, 2.0, 1.0, 'mp-1096051']]

In [10]:
len(substituted_list)

560

## 치환된 composition 으로 dataframe 만들기

In [11]:
def comp_to_form(tt):
    #tt = ['Li', '에스',  'O', 3.0, 2.0,  12.0]
    form = ''
    n_ele = int(len(tt)/2)
    for i in range(n_ele):
        form += str(tt[i]) + str(tt[n_ele+i])
    return form
def comp_to_ele(tt):
    #tt = ['Li', '에스',  'O', 3.0, 2.0,  12.0]
    form = ''
    n_ele = int(len(tt)/2)
    for i in range(n_ele):
        form += str(tt[i])# + str(tt[n_ele+i])
    return form

In [12]:
subcomp_mpid_df = pd.DataFrame()
sub_comp_list = []
mpid_list = []
reduced_form_list = []
element = []
for i in tqdm(substituted_list):
    my_comp_list=i[:-1]
    mpid=i[-1]
    reduced_form=Composition(comp_to_form(my_comp_list)).reduced_formula
#     ele_list = Composition(subcomp_mpid_df['reduced_formula'][i]).chemical_system.split('-')
    
    sub_comp_list.append(my_comp_list)
    mpid_list.append(mpid)
    reduced_form_list.append(reduced_form)
#     elements.append(tuple(sorted(ele_list)))
subcomp_mpid_df['sub_comp'] = sub_comp_list
subcomp_mpid_df['mpid'] = mpid_list
subcomp_mpid_df['reduced_formula'] = reduced_form_list
# subcomp_mpid_df['elements'] = elements
subcomp_mpid_df

  0%|          | 0/560 [00:00<?, ?it/s]

,sub_comp,mpid,reduced_formula
0,"[Li, S, Li, 1.0, 2.0, 1.0]",mp-1096051,LiS
1,"[Li, S, B, 1.0, 2.0, 1.0]",mp-1096051,LiBS2
2,"[Li, S, C, 1.0, 2.0, 1.0]",mp-1096051,LiCS2
3,"[Li, S, N, 1.0, 2.0, 1.0]",mp-1096051,LiS2N
4,"[Li, S, O, 1.0, 2.0, 1.0]",mp-1096051,LiS2O
...,...,...,...
555,"[Li, Te, S, 3.0, 5.0, 8.0]",mp-754811,Li3Te5S8
556,"[Li, I, S, 3.0, 5.0, 8.0]",mp-754811,Li3S8I5
557,"[Li, Ba, S, 3.0, 5.0, 8.0]",mp-754811,Ba5Li3S8
558,"[Li, Hg, S, 3.0, 5.0, 8.0]",mp-754811,Li3Hg5S8


In [13]:
subcomp_mpid_df.to_csv('./new_order_data/01_2_substitution_final.csv',index=False)

In [14]:
subcomp_mpid_df = pd.read_csv('./new_order_data/01_2_substitution_final.csv')

In [15]:
subcomp_mpid_df

,sub_comp,mpid,reduced_formula
0,"['Li', 'S', 'Li', 1.0, 2.0, 1.0]",mp-1096051,LiS
1,"['Li', 'S', 'B', 1.0, 2.0, 1.0]",mp-1096051,LiBS2
2,"['Li', 'S', 'C', 1.0, 2.0, 1.0]",mp-1096051,LiCS2
3,"['Li', 'S', 'N', 1.0, 2.0, 1.0]",mp-1096051,LiS2N
4,"['Li', 'S', 'O', 1.0, 2.0, 1.0]",mp-1096051,LiS2O
...,...,...,...
555,"['Li', 'Te', 'S', 3.0, 5.0, 8.0]",mp-754811,Li3Te5S8
556,"['Li', 'I', 'S', 3.0, 5.0, 8.0]",mp-754811,Li3S8I5
557,"['Li', 'Ba', 'S', 3.0, 5.0, 8.0]",mp-754811,Ba5Li3S8
558,"['Li', 'Hg', 'S', 3.0, 5.0, 8.0]",mp-754811,Li3Hg5S8


In [16]:
selected_structure_df = subcomp_mpid_df
selected_structure_df = pd.merge(df[['material_id','cif']],selected_structure_df,left_on='material_id',right_on='mpid',how='right').rename({'cif':'ref_cif'},axis=1)
selected_structure_df

,material_id,ref_cif,sub_comp,mpid,reduced_formula
0,mp-1096051,# generated using pymatgen\ndata_LiCd2Cu\n_sym...,"['Li', 'S', 'Li', 1.0, 2.0, 1.0]",mp-1096051,LiS
1,mp-1096051,# generated using pymatgen\ndata_LiCd2Cu\n_sym...,"['Li', 'S', 'B', 1.0, 2.0, 1.0]",mp-1096051,LiBS2
2,mp-1096051,# generated using pymatgen\ndata_LiCd2Cu\n_sym...,"['Li', 'S', 'C', 1.0, 2.0, 1.0]",mp-1096051,LiCS2
3,mp-1096051,# generated using pymatgen\ndata_LiCd2Cu\n_sym...,"['Li', 'S', 'N', 1.0, 2.0, 1.0]",mp-1096051,LiS2N
4,mp-1096051,# generated using pymatgen\ndata_LiCd2Cu\n_sym...,"['Li', 'S', 'O', 1.0, 2.0, 1.0]",mp-1096051,LiS2O
...,...,...,...,...,...
555,mp-754811,# generated using pymatgen\ndata_Li3Mn5O8\n_sy...,"['Li', 'Te', 'S', 3.0, 5.0, 8.0]",mp-754811,Li3Te5S8
556,mp-754811,# generated using pymatgen\ndata_Li3Mn5O8\n_sy...,"['Li', 'I', 'S', 3.0, 5.0, 8.0]",mp-754811,Li3S8I5
557,mp-754811,# generated using pymatgen\ndata_Li3Mn5O8\n_sy...,"['Li', 'Ba', 'S', 3.0, 5.0, 8.0]",mp-754811,Ba5Li3S8
558,mp-754811,# generated using pymatgen\ndata_Li3Mn5O8\n_sy...,"['Li', 'Hg', 'S', 3.0, 5.0, 8.0]",mp-754811,Li3Hg5S8


In [17]:
structure_substitution_input_list = [r.to_dict() for _, r in tqdm(selected_structure_df[['mpid','sub_comp','ref_cif']].iterrows())]
with open('./new_order_data/01_3_structure_substitution_input_list.pkl','wb') as f:
    pickle.dump(structure_substitution_input_list,f)

0it [00:00, ?it/s]

In [19]:
structure_substitution_input_list[0]

{'mpid': 'mp-1096051',
 'sub_comp': "['Li', 'S', 'Li', 1.0, 2.0, 1.0]",
 'ref_cif': "# generated using pymatgen\ndata_LiCd2Cu\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   10.86791741\n_cell_length_b   10.86791741\n_cell_length_c   10.86791741\n_cell_angle_alpha   120.69537374\n_cell_angle_beta   119.72607237\n_cell_angle_gamma   89.63746566\n_symmetry_Int_Tables_number   1\n_chemical_formula_structural   LiCd2Cu\n_chemical_formula_sum   'Li1 Cd2 Cu1'\n_cell_volume   904.66950162\n_cell_formula_units_Z   1\nloop_\n _symmetry_equiv_pos_site_id\n _symmetry_equiv_pos_as_xyz\n  1  'x, y, z'\nloop_\n _atom_site_type_symbol\n _atom_site_label\n _atom_site_symmetry_multiplicity\n _atom_site_fract_x\n _atom_site_fract_y\n _atom_site_fract_z\n _atom_site_occupancy\n  Li  Li0  1  0.00000000  0.00000000  0.00000000  1\n  Cd  Cd1  1  0.00000000  0.26158700  0.26158700  1\n  Cd  Cd2  1  0.00000000  0.73841300  0.73841300  1\n  Cu  Cu3  1  0.00000000  0.50000000  0.50000000  1\n"}

In [52]:
def test_apply(x):
    mpid = x['material_id']
    struc = Structure.from_str(x['ref_cif'],fmt='cif')
    initial_chemsys_list = (df[df['material_id'] == mpid]['formula']).iloc[0] # struc.composition.chemical_system.split('-')
    final_chemsys_list = literal_eval(x['sub_comp'])
    
    return [mpid,final_chemsys_list,struc]
test_df = selected_structure_df[:3]
test_df

,material_id,ref_cif,sub_comp,mpid,reduced_formula
0,mp-1096051,# generated using pymatgen\ndata_LiCd2Cu\n_sym...,"[Li, S, Li, 1.0, 2.0, 1.0]",mp-1096051,LiS
1,mp-1096051,# generated using pymatgen\ndata_LiCd2Cu\n_sym...,"[Li, S, B, 1.0, 2.0, 1.0]",mp-1096051,LiBS2
2,mp-1096051,# generated using pymatgen\ndata_LiCd2Cu\n_sym...,"[Li, S, C, 1.0, 2.0, 1.0]",mp-1096051,LiCS2


In [53]:
test_apply(test_df.iloc[0])

ValueError: malformed node or string: ['Li', 'S', 'Li', 1.0, 2.0, 1.0]

In [29]:
test_df[['material_id','ref_cif','sub_comp']].apply(test_apply)

KeyError: 'material_id'

In [10]:
selected_structure_df['ref_cif'][0]

"# generated using pymatgen\ndata_LiTiS2\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   6.43570339\n_cell_length_b   6.43570339\n_cell_length_c   6.43570390\n_cell_angle_alpha   31.44540514\n_cell_angle_beta   31.44540514\n_cell_angle_gamma   31.44540549\n_symmetry_Int_Tables_number   1\n_chemical_formula_structural   LiTiS2\n_chemical_formula_sum   'Li1 Ti1 S2'\n_cell_volume   64.39980772\n_cell_formula_units_Z   1\nloop_\n _symmetry_equiv_pos_site_id\n _symmetry_equiv_pos_as_xyz\n  1  'x, y, z'\nloop_\n _atom_site_type_symbol\n _atom_site_label\n _atom_site_symmetry_multiplicity\n _atom_site_fract_x\n _atom_site_fract_y\n _atom_site_fract_z\n _atom_site_occupancy\n  Li  Li0  1  0.50000000  0.50000000  0.50000000  1\n  Ti  Ti1  1  0.00000000  0.00000000  0.00000000  1\n  S  S2  1  0.74577900  0.74577900  0.74577900  1\n  S  S3  1  0.25422100  0.25422100  0.25422100  1\n"

In [10]:
len(structure_substitution_input_list)

26549208

In [9]:
def structure_substitution(row,df=df): # rowDataFramempid, reference structure, substituted chemical system dictionaly expected
    mpid = row['mpid']
    struc = Structure.from_str(row['ref_cif'],fmt='cif')
    initial_chemsys_list = (df[df['material_id'] == mpid]['formula']).iloc[0] # struc.composition.chemical_system.split('-')
    final_chemsys_list = literal_eval(row['sub_comp'])
    
    struc.replace_species(dict(zip(initial_chemsys_list[:3],final_chemsys_list[:3])))
    poscar = struc.to(fmt='poscar')
    return [mpid,final_chemsys_list,poscar]

In [10]:
def structure_substitution_in_file(row,df=df): # rowDataFramempid, reference structure, substituted chemical system dictionaly expected
    mpid = row['mpid']
    struc = Structure.from_str(row['ref_cif'],fmt='cif')
    initial_chemsys_list = (df[df['material_id'] == mpid]['formula']).iloc[0] # struc.composition.chemical_system.split('-')
    final_chemsys_list = literal_eval(row['sub_comp'])
    
    struc.replace_species(dict(zip(initial_chemsys_list[:3],final_chemsys_list[:3])))
    poscar = struc.to(fmt='poscar')
    with open('./new_order_data/01_3_structure_sub.txt','ab') as f:
#         f.write(str([mpid,final_chemsys_list,poscar])+',\n')
        pickle.dump(str([mpid,final_chemsys_list,poscar])+',\n',f)
#     return [mpid,final_chemsys_list,poscar]

In [2]:
with open('./new_order_data/01_3_structure_substitution_input_list.pkl','rb') as f:
    structure_substitution_input_list = pickle.load(f)

In [3]:
len(structure_substitution_input_list)

26549208

In [7]:
mpid = structure_substitution_input_list[0]['mpid']
st = Structure.from_str(structure_substitution_input_list[0]['ref_cif'],fmt='cif')
ini = (df[df['material_id'] == mpid]['formula']).iloc[0][:3]
fin = literal_eval(structure_substitution_input_list[0]['sub_comp'])
dict(zip(ini,fin))
ini

"{'L"

In [69]:
(df[df['material_id'] == mpid]['formula']).iloc[0]

['Li', 'Ti', 'S', 1.0, 1.0, 2.0]

In [11]:
print(structure_substitution(structure_substitution_input_list[0]))

ValueError: Can't parse Element or String from type <class 'str'>: {.

In [20]:
with open('./new_order_data/01_3_structure_sub.txt','a') as f:
    f.writelines(str(structure_substitution(structure_substitution_input_list[0]))+',\n')

In [ ]:
num_cores = multi.cpu_count() # 8

# memory = Memory('./data/structure_substitution_multiprocesser',verbose=0)
# structure_substitution_cached = memory.cache(structure_substitution)

with multi.Pool(num_cores) as pool:
    structure_substitution_result = list(tqdm(pool.imap(structure_substitution, structure_substitution_input_list)))

structure_substitution_result

0it [00:00, ?it/s]

/home/kjs4697/.local/lib/python3.8/site-packages/pymatgen/io/cif.py:1165: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))
/home/kjs4697/.local/lib/python3.8/site-packages/pymatgen/io/cif.py:1165: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))
/home/kjs4697/.local/lib/python3.8/site-packages/pymatgen/io/cif.py:1165: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))
/home/kjs4697/.local/lib/python3.8/site-packages/pymatgen/io/cif.py:1165: UserWarning: I

In [31]:
num_cores = multi.cpu_count() # 8

# memory = Memory('./data/structure_substitution_multiprocesser',verbose=0)
# structure_substitution_cached = memory.cache(structure_substitution)

with multi.Pool(4) as pool:
    tqdm(pool.map(structure_substitution_in_file, structure_substitution_input_list))



Process ForkPoolWorker-28:
Process ForkPoolWorker-25:
Process ForkPoolWorker-27:
Process ForkPoolWorker-26:
Traceback (most recent call last):
  File "/home/kjs4697/miniconda3/envs/dft/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/kjs4697/.local/lib/python3.8/site-packages/pymatgen/io/cif.py", line 1152, in get_structures
    s = self._get_structure(d, primitive, symmetrized)
  File "/home/kjs4697/miniconda3/envs/dft/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/kjs4697/miniconda3/envs/dft/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/kjs4697/miniconda3/envs/dft/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/kjs4697/miniconda3/envs/dft/lib/python3.8/multiprocessing/process.py", line 108,

KeyboardInterrupt: 

In [47]:
with open('./new_order_data/01_3_structure_sub.txt','wb') as f:
#     f.writelines('[\n')
    pickle.dump('[\n',f)
    
parmap.map(structure_substitution_in_file,structure_substitution_input_list[:200],pm_pbar=True)

with open('./new_order_data/01_3_structure_sub.txt','ab') as f:
#     f.writelines(']')
    pickle.dump(']',f)

  0%|          | 0/200 [00:00<?, ?it/s]

/home/kjs4697/.local/lib/python3.8/site-packages/pymatgen/io/cif.py:1165: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))
/home/kjs4697/.local/lib/python3.8/site-packages/pymatgen/io/cif.py:1165: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))
/home/kjs4697/.local/lib/python3.8/site-packages/pymatgen/io/cif.py:1165: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))
/home/kjs4697/.local/lib/python3.8/site-packages/pymatgen/io/cif.py:1165: UserWarning: I

In [56]:
with open('./new_order_data/01_3_structure_sub.txt','rb') as f:
#     f.writelines(']')
    te = pickle.load(f)
    te2 = pickle.load(f)

In [57]:
te

'[\n'

In [12]:
structure_substitution_in_file(structure_substitution_input_list[0])

In [ ]:
with open('./new_order_data/01_3_structure_sub.txt','w') as f:
    f.writelines('[\n')
    
parmap.map(structure_substitution_in_file,structure_substitution_input_list,pm_pbar=True,pm_processes=4)

with open('./new_order_data/01_3_structure_sub.txt','a') as f:
    f.writelines(']')

  0%|          | 0/26549208 [00:02<?, ?it/s]

/home/kjs4697/.local/lib/python3.8/site-packages/pymatgen/io/cif.py:1165: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))
/home/kjs4697/.local/lib/python3.8/site-packages/pymatgen/io/cif.py:1165: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))
/home/kjs4697/.local/lib/python3.8/site-packages/pymatgen/io/cif.py:1165: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))
/home/kjs4697/.local/lib/python3.8/site-packages/pymatgen/io/cif.py:1165: UserWarning: I

In [18]:
with open('./new_order_data/01_3_structure_sub.txt','rt') as f:
    ll = f.read()
len(literal_eval(ll))

200

In [16]:
literal_eval('[[1],\n[2]]')

[[1], [2]]

In [ ]:
structure_substitution_result_df = pd.DataFrame(structure_substitution_result,columns=['mpid','sub_comp','poscar'])
structure_substitution_result_df

In [ ]:
structure_substitution_final_df = pd.merge(selected_structure_df,structure_substitution_result_df,on=['mpid','sub_comp'],how='right')

In [ ]:
structure_substitution_final_df.to_csv('./new_order_data/01_4_structure_substitution_final.csv',index=False)